In [1]:
import os
import cv2
import openpyxl

def create_excel_file(filename):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Teachers"
    sheet.append(["Name", "Subject", "Face_Data", "Trained"])
    workbook.save(filename)

def create_teacher_dataset(teacher_name, subject, dataset_path):
    teacher_dir = os.path.join(dataset_path, teacher_name)
    os.makedirs(teacher_dir, exist_ok=True)
    return teacher_dir, subject

def capture_face_data(teacher_dir, teacher_name, teacher_subject, webcam_id=0, num_images=80):
    # Initialize face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Initialize video capture
    cap = cv2.VideoCapture(webcam_id)

    # Initialize sample counter
    sample_num = 0

    while sample_num < num_images:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        # Iterate over detected faces
        for (x, y, w, h) in faces:
            # Draw rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Save the detected face image
            face_img_path = os.path.join(teacher_dir, f"{teacher_name}_{sample_num}.png")
            cv2.imwrite(face_img_path, gray[y:y+h, x:x+w])

            # Increment sample counter
            sample_num += 1

            # Display frame
            cv2.imshow('Capturing Face Data', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video capture and destroy windows
    cap.release()
    cv2.destroyAllWindows()

    return sample_num

def update_excel(teacher_name, teacher_subject, success):
    workbook = openpyxl.load_workbook('teachers.xlsx')
    sheet = workbook.active

    for row in sheet.iter_rows(min_row=2, max_col=2, max_row=sheet.max_row):
        if row[0].value == teacher_name:
            row[1].value = teacher_subject
            if success:
                row[2].value = "✔"
            else:
                row[2].value = "✘"
            break
    else:
        # If the teacher's name is not found, add a new row
        sheet.append([teacher_name, teacher_subject, "✔" if success else "✘"])

    workbook.save('teachers.xlsx')

def main():
    # Create 'datasets' directory if it doesn't exist
    dataset_path = "datasets"
    os.makedirs(dataset_path, exist_ok=True)
    
     # Check if teachers.xlsx exists, if not, create it
    if not os.path.exists('teachers.xlsx'):
        create_excel_file("teachers.xlsx")
    
    # Ask for teacher's name and subject
    teacher_name = input("Enter the teacher's name: ")
    teacher_subject = input("Enter the teacher's subject: ")

    # Create directory for the teacher's dataset
    teacher_dir, teacher_subject = create_teacher_dataset(teacher_name, teacher_subject, dataset_path)

    # Capture face data for the teacher
    num_images_captured = capture_face_data(teacher_dir, teacher_name, teacher_subject)

    if num_images_captured == 80:
        update_excel(teacher_name, teacher_subject, True)
        print(f"Face data captured successfully for {teacher_name}.")
    else:
        update_excel(teacher_name, teacher_subject, False)
        print("Face data capture unsuccessful.")

if __name__ == "__main__":
    main()


Enter the teacher's name: daksh
Enter the teacher's subject: maths
Face data captured successfully for daksh.
